In [1]:
%load_ext autoreload
%autoreload 2

Internal Imports

In [2]:
# Find ".env" file and add the package to $PATH
import os, sys
import typing as t
from typing import Dict, TypeAlias, Any
from dotenv import find_dotenv
sys.path.append(os.path.dirname(find_dotenv()))

# Use local package for modularity
import emotion_analysis as ea
from emotion_analysis.data.dataset import ECACDataset
from emotion_analysis.data.types import TrainSplit, DataSplit, SubTask
from emotion_analysis.data.transform import DataTokenize, DataTransform, DataCollator
from emotion_analysis.model.emotion_cause_text import load_text_model
from emotion_analysis.model.emotion_cause_text import EmotionCauseTextModel

JAX Backend:  gpu
JAX Version:  0.4.23
Python:  3.11.0 (main, Oct  4 2023, 22:00:02) [GCC 13.2.1 20230801]
System:  posix.uname_result(sysname='Linux', nodename='archlinux', release='6.6.10-arch1-1', version='#1 SMP PREEMPT_DYNAMIC Fri, 05 Jan 2024 16:20:41 +0000', machine='x86_64')


External Imports

In [3]:
import jax
import jax.numpy as jnp
import jax.random as jrm
from jax import Array
import jax.tree_util as tree_util
from jax.tree_util import tree_structure
from jax.typing import ArrayLike, DTypeLike
import flax
import flax.linen as nn
import numpy as np
import optax as opt
import mlflow as mlf
from transformers import RobertaConfig, FlaxRobertaModel, RobertaTokenizerFast
from transformers import PretrainedConfig
from torch.utils.data import Dataset, DataLoader, Subset, random_split
import pandas as pd
from collections import defaultdict
from itertools import chain

In [15]:
# Configuration
batch_size = 32
max_conv_len = 33
max_uttr_len = 93
subtask: SubTask = '2'

# Load pretrained model along with its tokenizer
text_encoder_pretrained = load_text_model()
tokenizer = text_encoder_pretrained.tokenizer
tokenize = DataTokenize(tokenizer, max_seq_len=max_uttr_len, padding='max_length')

# Load data subsets
ds_train = ECACDataset(
    data_dir=ea.DATA_DIR,
    subtask=subtask,
    split='train',
)
ds_test = ECACDataset(
    data_dir=ea.DATA_DIR,
    subtask=subtask,
    split='test',
)

# Data preprocessors
transform: Dict[TrainSplit, DataTransform] = {
    'train': DataTransform.from_data(ds_train, tokenize),
    'valid': DataTransform.from_data(ds_train, tokenize),
    'test': DataTransform.from_data(ds_test, tokenize),
}
collator: Dict[TrainSplit, DataCollator] = tree_util.tree_map(DataCollator, transform)

# Split training into train-valid subsets
ds_train, ds_valid = random_split(ds_train, [0.75, 0.25])
num_classes: int = len(ds_test.emotions)

# Split into train-valid-test
# Train: drop_last=True to avoid JAX graph recompilation
dataloader: t.Dict[TrainSplit, DataLoader[t.Dict[str, Array]]] = {
    'train': DataLoader(ds_train, batch_size, False, collate_fn=collator['train'], drop_last=True),
    'valid': DataLoader(ds_valid, batch_size, False, collate_fn=collator['valid']),
    'test': DataLoader(ds_test, batch_size, False, collate_fn=collator['test']),
}

In [6]:
# Generate a PR key for init
key = jrm.key(ea.SEED)
key, init_key = jrm.split(key, 2)

# Generate fake data to "imitate" a batch
fake_input_ids = jnp.zeros((batch_size, max_conv_len * max_conv_len))
fake_attn_mask = jnp.zeros_like(fake_input_ids)
fake_batch: Any = dict(input_ids=fake_input_ids, attention_mask=fake_attn_mask)

# Initialize the model with random params
ec_model = EmotionCauseTextModel(text_encoder=text_encoder_pretrained.module, num_classes=num_classes)
vars = ec_model.init(init_key, **fake_batch)
params = vars['params']

# Transfer the pretrained weights 
params['text_encoder'] = text_encoder_pretrained.params

In [7]:
# Train only the classifier for now...
should_freeze = lambda p, _: 'frozen' if 'text_encoder' in p else 'trainable'
param_labels = flax.traverse_util.path_aware_map(should_freeze, params)
tx = opt.multi_transform({ 'trainable': opt.adamw(2e-4), 'frozen': opt.set_to_zero() }, param_labels)
opt_state = tx.init(params)